# AFP Project

WRDS data

IMPORTANT: change user name to your own wrds username. Remeber to do DUO 2FA on WRDS: https://wrds-www.wharton.upenn.edu/pages/about/log-in-to-wrds-using-two-factor-authentication/

In [1]:
import wrds
import datetime as dt
import pandas as pd
import numpy as np
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
if 'conn' in locals():
    print("WRDS connection already open!")
else:
    conn = wrds.Connection(wrds_username='chunnijiang')

# set sample date range
begdate = '01/01/2015'
enddate = '12/30/2022'

Loading library list...
Done


In [3]:
#####################################################
# Calculate Trades, Flows and Turnover Ratios       #
# November 2018                                     #
# Input  - Thomson-Reuters 13F Data (TR-13F)        #
#          S34TYPE3 Holdings data                   #
#          S34TYPE1 data                            #
#          CRSP Stock File data                     #
# Output - TRADES data with detailed Buys & Sales   #
#          by each institution                      #
#          AGGREGATES data with Total Buys & Sales  #
#          Assets, Flows and Turnover Measures      #
#####################################################
# sql to extract data from CRSP
price = conn.raw_sql(f"""
                      select permno, date, cfacpr, cfacshr, shrout, prc, ret
                      from crsp.msf 
                      where date between '{begdate}' and '{enddate}'
                      """, date_cols=['date']) 

# change variable format to int
price[['permno']]= price[['permno']].astype(int)

# get month and quarter-end dates
price['mdate'] = price['date'] + pd.offsets.MonthEnd(0)
price['qdate'] = price['date'] + pd.offsets.QuarterEnd(0)

# calculate adjusted price, total shares and market cap
price['p'] = price['prc'].abs() / price['cfacpr'] # price adjusted
price['tso'] = price['shrout'] * price['cfacshr'] * 1e3 # total shares out adjusted

# Keep only the records with shrout>0
price = price[price['shrout'] > 0]

# keep only relevant columns
price = price[['permno','mdate','qdate','date','cfacshr', 'p', 'tso','ret']]

# create log return for computing quarterly compounded return
price['ret'] = price['ret'].fillna(0)
price['logret'] = np.log(1+price['ret'])

qret = price.groupby(['permno','qdate'])['logret'].sum().reset_index()
qret['qret']=np.exp(qret['logret'])-1

# shift qdate by one quarter to make qret next quarter return
qret['qdate']=qret['qdate']+pd.offsets.QuarterEnd(-1)
qret = qret.drop(columns=['logret'], axis=1)

# Finalize CRSP Subset - Keep Quarterly Observations 
# And Add Forward Quartely Returns
price = price[price['qdate']==price['mdate']]
price = price[['qdate','permno','cfacshr','p','tso']]

price = pd.merge(price, qret, how='left', on=['permno','qdate'])

In [4]:
price.head()

,qdate,permno,cfacshr,p,tso,qret
0,2015-03-31,10001,1.0,9.96,10452000.0,0.034137
1,2015-06-30,10001,1.0,10.30,10493000.0,-0.120357
2,2015-09-30,10001,1.0,8.94,10500000.0,-0.138579
3,2015-12-31,10001,1.0,7.45,10505000.0,0.048323
4,2016-03-31,10001,1.0,7.81,10508000.0,-0.095798


In [5]:
######################################
# Step 1                             #
# Merge TR13F S34Type1 & 4Type3 sets #
######################################

# First keep first vintage with holdings data for each rdate-mgrno combination
fst_vint = conn.raw_sql("""
                      select rdate, fdate, mgrno, mgrname
                      from tfn.s34type1 
                      """, date_cols=['rdate','fdate'])

# Keep first vintage with holding data for each mgrno-rdate combo
min_fdate = fst_vint.groupby(['mgrno','rdate'])['fdate'].min().reset_index()

# Merge back with the fst_vint data to keep only the first vintage records
fst_vint = pd.merge(fst_vint, min_fdate, how='inner', on=['mgrno','rdate','fdate'])

# Sort by mgrno and rdate and create lag_rdate to calculate gap
fst_vint = fst_vint.sort_values(['mgrno', 'rdate'])
fst_vint['lag_rdate']=fst_vint.groupby(['mgrno'])['rdate'].shift(1)

fst_vint.dropna(inplace=True)
def diff_q(a: pd.Period, b: pd.Period) -> int:
    return (a - b).n

fst_vint['qtr'] = fst_vint.apply(lambda row: diff_q(row['rdate'].to_period('Q'), row['lag_rdate'].to_period('Q')), axis=1)
fst_vint['qtr'] = fst_vint['qtr'].fillna(-1)

fst_vint['first_report'] = ((fst_vint.qtr.isnull()) | (fst_vint.qtr>=2))
fst_vint = fst_vint.drop(['qtr'],axis=1)

# Last report by manager or missing 13F reports in the next quarter(s)
fst_vint = fst_vint.sort_values(['mgrno','rdate'], ascending=[True, False])
fst_vint['lead_rdate'] = fst_vint.groupby(['mgrno'])['rdate'].shift(1)

# Number of quarters gap between lead_rdate and rdate
fst_vint.dropna(inplace=True)
fst_vint['qtr'] = fst_vint.apply(lambda row: diff_q(row['lead_rdate'].to_period('Q'), row['rdate'].to_period('Q')), axis=1)
fst_vint['qtr'] = fst_vint['qtr'].fillna(-1)

# label last_report flag
fst_vint['last_report'] = ((fst_vint.qtr.isnull()) | (fst_vint.qtr>=2))
fst_vint = fst_vint.drop(['qtr'],axis=1)

fst_vint = fst_vint[(fst_vint['rdate']<=enddate) & (fst_vint['rdate']>=begdate)]\
                    .drop(['lag_rdate','lead_rdate'], axis=1)

In [8]:
fst_vint = fst_vint[fst_vint['mgrname'].astype(str).str.endswith(('LP', 'L.P.', 'LLP', 'L.L.P.', 'LLC', 'L.L.C.'))]
fst_vint
# rdate: holdings report date
# fdate: vintage date
# mgrno: institution code

,rdate,fdate,mgrno,mgrname,first_report,last_report
387367,2022-09-30,2022-09-30,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False
380612,2022-06-30,2022-06-30,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False
373862,2022-03-31,2022-03-31,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False
367017,2021-12-31,2021-12-31,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False
361005,2021-09-30,2021-09-30,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False
...,...,...,...,...,...,...
256417,2016-03-31,2016-03-31,95110.0,PHOENIX/ZWEIG ADVISERS LLC,False,False
252272,2015-12-31,2015-12-31,95110.0,PHOENIX/ZWEIG ADVISERS LLC,False,False
248065,2015-09-30,2015-09-30,95110.0,PHOENIX/ZWEIG ADVISERS LLC,False,False
244059,2015-06-30,2015-06-30,95110.0,PHOENIX/ZWEIG ADVISERS LLC,False,False


In [10]:
######################################
# Step 2                             #
# Extract Holdings and Adjust Shares #
######################################

# Fdate - Vintage Date - is used in shares adjustment

s34type3 = conn.raw_sql(f"""
                      select fdate, mgrno, cusip, shares
                      from tfn.s34type3
                      where fdate between '{begdate}' and '{enddate}'
                      """, date_cols=['fdate']) 


In [11]:
s34type3.head()

,fdate,mgrno,cusip,shares
0,2015-03-31,110.0,00206R10,59438.0
1,2015-03-31,110.0,00282410,8700.0
2,2015-03-31,110.0,00287Y10,12200.0
3,2015-03-31,110.0,01717510,1594.0
4,2015-03-31,110.0,02209S10,103906.0


In [12]:
holdings_v1 = pd.merge(fst_vint, s34type3, how='inner', on=['fdate','mgrno'] )
# Map 13F's historical cusip to CRSP's permno information
crsp = conn.raw_sql("""
                    select distinct permno, ncusip
                    from crsp.msenames
                    where ncusip != ''
                    """)

holdings_v2 = pd.merge(holdings_v1, crsp, how='inner', left_on='cusip', right_on='ncusip')
holdings_v2 = holdings_v2.drop(['mgrname','cusip','ncusip'], axis=1)

In [13]:
holdings_v2.head()

,rdate,fdate,mgrno,first_report,last_report,shares,permno
0,2022-09-30,2022-09-30,127.0,False,False,511200.0,80412
1,2022-06-30,2022-06-30,127.0,False,False,587020.0,80412
2,2021-12-31,2021-12-31,127.0,False,False,862390.0,80412
3,2021-09-30,2021-09-30,127.0,False,False,892390.0,80412
4,2021-06-30,2021-06-30,127.0,False,False,994365.0,80412


In [14]:
######################################
# Step 3                             #
# Adjust Shares Using CRSP CFACSHR   #
# Align at Vintage Dates             #
######################################

holdings = pd.merge(holdings_v2, price[['qdate','permno','cfacshr']], \
                    how='inner', left_on=['permno','fdate'], right_on=['permno','qdate'])

# Calculate Adjusted Shares
holdings['shares_adj']=holdings['shares']*holdings['cfacshr']
holdings=holdings.drop(['qdate','cfacshr','fdate'], axis=1)

# Sanity Checks for Duplicates - Ultimately, Should be 0 Duplicates
holdings = holdings.drop_duplicates(subset=['mgrno','permno','rdate'])

# Keep only observations with shares_adj>0
holdings = holdings[holdings['shares_adj']>0]

In [15]:
holdings.head()
# need 2

,rdate,mgrno,first_report,last_report,shares,permno,shares_adj
0,2022-09-30,127.0,False,False,511200.0,80412,511200.0
1,2022-09-30,6132.0,False,False,20422.0,80412,20422.0
2,2022-09-30,7049.0,False,False,5588.0,80412,5588.0
3,2022-09-30,7524.0,False,False,3577.0,80412,3577.0
4,2022-09-30,7633.0,False,False,7241.0,80412,7241.0


In [16]:
######################################
# Step 4                             #
# Calculate Institutional Trades     #
# Security-by-Security               #
# trade>0 -> Buy vs trade<0 -> Sale  #
# buysale variable for trade types:  #
#     1 = Initiating Buys            #
#     2 = Incremental (Regular) Buys #
#    -1 = Terminating Sales          #
#    -2 = Regular Sales              #
######################################

t1 = holdings.sort_values(['mgrno','permno','rdate'])

# create  phrdate pshares_adj trade and lead lag permno information

# previous holding quarter
t1['phrdate'] = t1.groupby(['mgrno','permno'])['rdate'].shift(1)
# previous quarter shares
t1['pshares_adj'] = t1.groupby(['mgrno','permno'])['shares_adj'].shift(1)
# trade as difference in current and previous quarter shares
t1['trade']=t1['shares_adj'] - t1['pshares_adj']

In [17]:
t1.head()
# pshares_adj: previous adjusted shares
# shares_adj
# trade: change in adjusted shares

,rdate,mgrno,first_report,last_report,shares,permno,shares_adj,phrdate,pshares_adj,trade
262564,2018-09-30,127.0,False,False,220707.0,12373,220707.0,NaT,NaN,NaN
262475,2018-12-31,127.0,False,False,290920.0,12373,290920.0,2018-09-30,220707.0,70213.0
262376,2019-03-31,127.0,False,False,269088.0,12373,269088.0,2018-12-31,290920.0,-21832.0
262271,2019-06-30,127.0,False,True,204225.0,12373,204225.0,2019-03-31,269088.0,-64863.0
232592,2015-03-31,127.0,False,False,1502354.0,12513,1502354.0,NaT,NaN,NaN


In [18]:
t1.isnull().sum()

rdate                 0
mgrno                 0
first_report          0
last_report           0
shares                0
permno                0
shares_adj            0
phrdate         1990962
pshares_adj     1990962
trade           1990962
dtype: int64

In [19]:
t1.dropna(inplace=True)

In [20]:
# quarter gap
t1['qtrgap'] = t1.apply(lambda row: diff_q(row['rdate'].to_period('Q'), row['phrdate'].to_period('Q')), axis=1)

# lag permno for determining first permno
t1['lpermno'] = t1['permno'].shift(1)

# lead permno for determining last permno
t1['npermno'] = t1['permno'].shift(-1)

In [22]:
t1.to_parquet('../data/holdings_hf.parquet.gzip', compression='gzip')

In [23]:
t1.head()

,rdate,mgrno,first_report,last_report,shares,permno,shares_adj,phrdate,pshares_adj,trade,qtrgap,lpermno,npermno
262475,2018-12-31,127.0,False,False,290920.0,12373,290920.0,2018-09-30,220707.0,70213.0,1,NaN,12373.0
262376,2019-03-31,127.0,False,False,269088.0,12373,269088.0,2018-12-31,290920.0,-21832.0,1,12373.0,12373.0
262271,2019-06-30,127.0,False,True,204225.0,12373,204225.0,2019-03-31,269088.0,-64863.0,1,12373.0,12513.0
232546,2015-06-30,127.0,False,False,1517954.0,12513,1517954.0,2015-03-31,1502354.0,15600.0,1,12373.0,12513.0
232501,2015-09-30,127.0,False,False,1387792.0,12513,1387792.0,2015-06-30,1517954.0,-130162.0,1,12513.0,12513.0


In [25]:
import pyarrow.parquet as pq

# Specify the path to your parquet.gzip file
parquet_file_path = 't1.parquet.gzip'

# Read the Parquet file using pyarrow
table = pq.read_table(parquet_file_path)

# Convert the table to a Pandas DataFrame if needed
df = table.to_pandas()

# Now, you can work with the DataFrame (df)
print(df.head())

FileNotFoundError: t1.parquet.gzip

In [26]:
# List conditions 
cond1 = (t1.permno!=t1.lpermno)
cond1_1 = (t1.permno!=t1.lpermno) & (~t1.first_report)
cond2_1 = (t1.permno==t1.lpermno) & (~t1.first_report) & (t1.qtrgap==1)
cond2_2 = (t1.permno==t1.lpermno) & (~t1.first_report) & (t1.qtrgap!=1)

# Assign modtrade value based on the conditions listed above
t1['modtrade'] = t1['trade']
t1.loc[cond1, 'modtrade'] = np.nan
t1.loc[cond1_1, 'modtrade'] = t1.loc[cond1_1, 'shares_adj']
t1.loc[cond2_1, 'modtrade'] = t1.loc[cond2_1, 'trade']
t1.loc[cond2_2, 'modtrade'] = t1.loc[cond2_2, 'shares_adj']

# Assign buysale value based on the conditions
t1.loc[cond1_1, 'buysale'] = 1
t1.loc[cond2_1, 'buysale'] = 2 * np.sign(t1.loc[cond2_1, 'trade'])
t1.loc[cond2_2, 'buysale'] = 1.5

# Adjusting rdate for terminating sales records
t1['rdate'] = np.where(t1['buysale']==-1, t1['rdate'] + pd.offsets.QuarterEnd(1), t1['rdate'])

# Focusing on cases of intermediate sales - with gaps > 1 qtr 
# Then need to split transaction into 2
t2=t1[(t1.buysale==1.5)]
t2['rdate'] = t2['phrdate']+pd.offsets.QuarterEnd(1)

t2['buysale'] = -1
t2['modtrade'] = -t2['pshares_adj']

# Go back to modify the t1 dataset with buysale variable labeled as 1.5
t1['buysale'] = np.where(t1['buysale']==1.5, 1, t1['buysale'])

# handle terminating sales
t3 = t1[(t1.permno != t1.npermno) & (t1.last_report == False)]

t3['rdate'] = t3['rdate']+pd.offsets.QuarterEnd(1)
t3['modtrade'] = -t3['shares_adj']
t3['buysale'] = -1

# Append t1 t2 and t3 to create the complete trades output
trades = pd.concat([t1, t2, t3])
trades = trades[(trades.modtrade != 0) & (trades.modtrade.notna()) & (trades.buysale.notna())]
trades = trades[['rdate','mgrno','permno','modtrade','buysale']]\
.rename(columns={'modtrade':'trade'})

In [27]:
######################################
# Step 6                             #
# Calculate Assets, Total Buys, and  #
# Total Sales per Institution Each   #
# Quarter                            #
######################################

# Get Total Assets and Portfolio Returns from Holdings  
# Assuming Buys and Sales Are Executed at Calendar Quarter Ends

_holdings = holdings[['mgrno','permno','rdate','shares_adj']]
_price = price[['permno','qdate','p','qret']]
_assets = pd.merge(_holdings, _price, \
                  how='inner', left_on=['permno','rdate'], right_on=['permno','qdate'])

# create intermediate variables before summing by manager and quarter

# dollar held in each stock per quarter per manager
_assets['hold_per_stock'] = _assets['shares_adj']*_assets['p']/1000000
# dollar held projected to next quarter assuming same shares held
_assets['next_value'] = _assets['shares_adj']*_assets['p']*_assets['qret']
# dollar held curent quarter
_assets['curr_value'] = _assets['shares_adj']*_assets['p']

_assets = _assets.sort_values(['mgrno','rdate'])

# sum values across different stocks by a manager in a quarter
hold_per_stock = _assets.groupby(['mgrno','rdate'])['hold_per_stock'].sum().reset_index()
next_qtr = _assets.groupby(['mgrno','rdate'])['next_value'].sum().reset_index()
curr_qtr = _assets.groupby(['mgrno','rdate'])['curr_value'].sum().reset_index()

# Forward portfolio return
pret = pd.merge(next_qtr, curr_qtr, how='inner', on=['mgrno','rdate'])
pret['pret'] = pret['next_value']/pret['curr_value']
pret = pret[['mgrno','rdate','pret']]

assets = pd.merge(hold_per_stock, pret, how='inner',on=['mgrno','rdate'])

# Total portfolio Assets
assets = assets.rename(columns={'hold_per_stock':'assets'})

# Aggregate Total Buys and Sales per Institution Every Quarter 

# create intermediate variables first
_flows = pd.merge(trades, _price, how='inner', \
                  left_on=['permno','rdate'], right_on=['permno','qdate'])

# dollar amount buys per stock by a manager in a quarter
_flows['tbuys']  = _flows['trade']*(_flows['trade']>0) * _flows['p']/1000000
# dollar amount sells per stock by a manager in a quarter
_flows['tsales'] = (-1)*_flows['trade']*(_flows['trade']<0) * _flows['p']/1000000
# net gain from trades per stock 
_flows['tgain']  = _flows['trade']*_flows['p']*_flows['qret']/1000000

# sum values across different stocks by a manager in a quarter
tbuys  = _flows.groupby(['mgrno','rdate'])['tbuys'].sum().reset_index()
tsales = _flows.groupby(['mgrno','rdate'])['tsales'].sum().reset_index()
tgain  = _flows.groupby(['mgrno','rdate'])['tgain'].sum().reset_index()

ttran = pd.merge(tbuys, tsales, how='inner', on =['mgrno','rdate'])

# Flows dataframe
flows = pd.merge(ttran, tgain, how='inner',on=['mgrno','rdate'])

In [28]:
######################################
# Step 7                             #
# Calculate Net Flows and Turnover   #
######################################

fst_vint = fst_vint.sort_values(['mgrno','rdate'])
fst_vint = fst_vint.drop_duplicates(subset=['mgrno','rdate'])

# inner join fst_vint and assets dataframes then left join with flows
_agg1 = pd.merge(fst_vint, assets, how='inner', on =['mgrno','rdate'])
_agg1 = _agg1.drop(['fdate'], axis=1)
aggregates = pd.merge(_agg1, flows, how='left', on=['mgrno','rdate'])

# assets compound value
aggregates['assets_comp'] = aggregates['assets']*(1+aggregates['pret'])
aggregates = aggregates.sort_values(['mgrno','rdate'])
# lag asset compound value
aggregates['lassets_comp'] = aggregates.groupby(['mgrno'])['assets_comp'].shift(1)
# lag asset value
aggregates['lassets'] = aggregates.groupby(['mgrno'])['assets'].shift(1)

# Trade Returns = Returns on Purchases - Forgone Returns on Sales
aggregates['tgainret'] = aggregates['tgain']/(aggregates['tbuys'] + aggregates['tsales'])
aggregates['netflows'] = aggregates['assets'] - aggregates['lassets_comp']

# Three Types of Turnover Measures

# Carhart (1997) Turnover Definition
aggregates['turnover1'] = \
(aggregates[['tbuys', 'tsales']].min(axis=1)) / (aggregates[['assets', 'lassets']].mean(axis=1))

# Adding Back Net Flows and Redemptions
aggregates['turnover2'] = \
(aggregates[['tbuys', 'tsales']].min(axis=1) + aggregates['netflows'].abs().fillna(0)) \
/ aggregates['lassets']

# or, Alternatively
aggregates['turnover3'] = \
(aggregates['tbuys'].fillna(0)+aggregates['tsales'].fillna(0)\
-aggregates['netflows'].abs().fillna(0))\
/ aggregates['lassets']

# Assign missing values for first_report records
aggregates['netflows']=np.where(aggregates['first_report'], np.nan, aggregates['netflows'])
aggregates['tgainret']=np.where(aggregates['first_report'], np.nan, aggregates['tgainret'])
aggregates['turnover1']=np.where(aggregates['first_report'], np.nan, aggregates['turnover1'])
aggregates['turnover2']=np.where(aggregates['first_report'], np.nan, aggregates['turnover2'])
aggregates['turnover3']=np.where(aggregates['first_report'], np.nan, aggregates['turnover3'])

aggregates = aggregates.drop(['assets_comp', 'lassets_comp', 'lassets'], axis=1)
aggregates.to_parquet('aggregate_df.parquet.gzip', compression='gzip')
####################
# Close connection #
####################
conn.close()

######################################
##########  End of Program  ##########
######################################

In [29]:
aggregates.head()
# need3

,rdate,mgrno,mgrname,first_report,last_report,assets,pret,tbuys,tsales,tgain,tgainret,netflows,turnover1,turnover2,turnover3
0,2015-03-31,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False,4553.577546,-0.097883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-06-30,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False,4102.561959,0.036171,4051.512488,0.000000,146.225932,0.036092,-5.295758,0.000000,0.001163,0.888580
2,2015-09-30,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False,4064.568036,0.084079,136.028707,411.538853,-22.560811,-0.041202,-186.386535,0.033311,0.078589,0.088038
3,2015-12-31,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False,4502.353836,0.060149,641.889624,270.084462,42.346158,0.046434,96.041597,0.063053,0.090077,0.200743
4,2016-03-31,127.0,"AEW CAPITAL MANAGEMENT, L.P.",False,False,4568.146306,0.046889,59.260200,189.999783,-4.934305,-0.019796,-205.019623,0.013067,0.058698,0.009826


In [30]:
aggregates.to_parquet('../data/hf_netflow.parquet.gzip', compression='gzip')